In [1]:
import SimpleITK as sitk
from scipy.io import loadmat, savemat
import numpy as np

def resample(ori_img:sitk.Image, scale, resamplemethod=sitk.sitkNearestNeighbor) -> sitk.Image:
    ori_Size = ori_img.GetSize()
    ori_Spacing = ori_img.GetSpacing()   


    if isinstance(scale, list) and len(scale) == 3:
        target_Size = tuple([int(ori_Size[idx]*scale[idx]) for idx in range(3)])
        target_Spacing = tuple([ori_Spacing[idx]/(target_Size[idx]/ori_Size[idx]) for idx in range(3)])
    else:
        target_Size = tuple([int(size*scale) for size in ori_Size])
        target_Spacing = tuple([ori_Spacing[idx]/(target_Size[idx]/ori_Size[idx]) for idx in range(3)])
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(ori_img)  

    resampler.SetSize(target_Size)		
    resampler.SetOutputSpacing(target_Spacing)
    if resamplemethod == sitk.sitkNearestNeighbor:
        resampler.SetOutputPixelType(sitk.sitkUInt8)   # 近邻插值用于mask的，保存uint8
    else:
        resampler.SetOutputPixelType(sitk.sitkInt16)  # 线性插值用于PET/CT/MRI之类的，保存float32
    resampler.SetTransform(sitk.Transform(3, sitk.sitkIdentity))    
    resampler.SetInterpolator(resamplemethod)
    return resampler.Execute(ori_img) 

In [36]:
brain_image = sitk.ReadImage("B0map/brain3D_0.5.nii.gz")
print(brain_image.GetSize())
print(brain_image.GetOrigin())
print(brain_image.GetSpacing())


brain_image.SetOrigin((-90.5, -90.5, -108.5))
sitk.WriteImage(brain_image, 'B0map/brain3D_0.5_rot.nii.gz')

(362, 362, 434)
(0.0, 0.0, 0.0)
(0.5, 0.5, 0.5)


In [46]:
B0_image = sitk.ReadImage("B0map/source/B0map_hertz.nii.gz")
brain_image = sitk.ReadImage("B0map/source/brain.nii.gz")

B0_rot = sitk.GetImageFromArray(np.transpose(sitk.GetArrayFromImage(B0_image), (1, 2,0)))
B0_rot.SetSpacing(B0_image.GetSpacing())
B0_rot.SetOrigin((-80, -128, -128))
sitk.WriteImage(B0_rot, 'B0map/B0_rot.nii.gz')

brain_rot = sitk.GetImageFromArray(np.transpose(sitk.GetArrayFromImage(brain_image), (1, 2,0)))
brain_rot.SetSpacing(brain_image.GetSpacing())
brain_rot.SetOrigin((-80, -128, -128))
sitk.WriteImage(brain_rot, 'B0map/brain_rot.nii.gz')

In [3]:
B0_img = sitk.ReadImage("B0map/B02brain3D_masked.nii.gz") # 0.5 mm iso 
median_filter = sitk.MedianImageFilter()
median_filter.Execute(B0_img)

<SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x000002A25A6F6C10> >

In [4]:
B0_img = sitk.ReadImage("B0map/B02brain3D_masked.nii.gz") # 0.5 mm iso 
median_filter = sitk.MedianImageFilter()
B0_img = median_filter.Execute(B0_img)

B0_img.SetSpacing((0.5, 0.5, 0.5))

print(B0_img.GetSize())
print(B0_img.GetSpacing())
out_img = resample(ori_img=B0_img, scale=0.5, resamplemethod=sitk.sitkLinear)
print(out_img.GetSize())
print(out_img.GetSpacing())  

sitk.WriteImage(out_img, 'brain3D_B0map_{}.nii.gz'.format(out_img.GetSpacing()[0]))
savemat('brain3D_B0map_{}.mat'.format(out_img.GetSpacing()[0]), {'data': sitk.GetArrayFromImage(out_img)}, do_compression=True)

(362, 362, 434)
(0.5, 0.5, 0.5)
(181, 181, 217)
(1.0, 1.0, 1.0)


In [4]:
B0_img = sitk.ReadImage("B0map/B02brain3D_masked.nii.gz") # 0.5 mm iso 


B0_img.SetSpacing((0.5, 0.5, 0.5))

print(B0_img.GetSize())
print(B0_img.GetSpacing())
out_img = resample(ori_img=B0_img, scale=0.5, resamplemethod=sitk.sitkLinear)
print(out_img.GetSize())
print(out_img.GetSpacing())  

median_filter = sitk.MedianImageFilter()
median_filter.SetRadius(4)
out_img = median_filter.Execute(out_img)

sitk.WriteImage(out_img, 'brain3D_B0map_{}_median_filter.nii.gz'.format(out_img.GetSpacing()[0]))
savemat('brain3D_B0map_{}_median_filter.mat'.format(out_img.GetSpacing()[0]), {'data': sitk.GetArrayFromImage(out_img)}, do_compression=True)

(362, 362, 434)
(0.5, 0.5, 0.5)
(181, 181, 217)
(1.0, 1.0, 1.0)
